## 1. Import required Libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import sqlite3
from bs4 import BeautifulSoup

## 2. Initialize Particulars

In [3]:
url = 'https://en.wikipedia.org/wiki/Forbes_Global_2000'
table_attr = ["Company_Name", "Sector", "Headquarters", "Sales_in[$B]", "Profit_in[$B]",\
              "Asset_in[$B]", "Market_Value_in[$B]", "Overall_Rank"]
tranformed_data_path = './Top_Companies_by_Sector_data.csv'

## 3. Extract the data from the Website

In [4]:
html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')
    
df = pd.DataFrame(columns=table_attr)
table = data.find_all('tbody') 
rows = table[7].find_all('tr')# the seventh table on the website

for row in rows:
    col = row.find_all('td')
    if len(col) != 0:
        data_dict = {"Company_Name":col[1].contents[0],
                     "Sector":col[0].contents[0],
                     "Headquarters":col[2].contents[0],
                     "Sales_in[$B]":col[4].contents[0],
                     "Profit_in[$B]":col[5].contents[0],
                     "Asset_in[$B]":col[6].contents[0],
                     "Market_Value_in[$B]":col[7].contents[0],
                     "Overall_Rank":col[3].contents[0]
                        }
        df1 = pd.DataFrame(data_dict, index=[0])
        df = pd.concat([df,df1], ignore_index=True)
#There is a \n in market value column, that needs to be removed
MV = list(df['Market_Value_in[$B]'])
Modified_MV = [float(''.join(cell.split('\n'))) for cell in MV]
df['Market_Value_in[$B]'] = Modified_MV

#Remove the 0 in the beginning represent # in data from each cell in the overall rank column
OR = list(df['Overall_Rank'])
Modified_OR = [int(str(cell)[1:]) for cell in OR]
df['Overall_Rank'] = Modified_OR

In [5]:
print("Shape of the Dataframe: ", df.shape)
df.head()

Shape of the Dataframe:  (81, 8)


,Company_Name,Sector,Headquarters,Sales_in[$B],Profit_in[$B],Asset_in[$B],Market_Value_in[$B],Overall_Rank
0,WPP,Advertising,London,18.7,1.8,42.4,30.1,301
1,Boeing,Aerospace,Chicago,96.1,5.2,94.4,85.3,76
2,United Parcel Service,Air Courier,Sandy Springs,58.1,4.8,38.3,93.3,146
3,Delta Air Lines,Airline,Atlanta,40.5,4.7,53.3,34.4,170
4,Alcoa,Aluminum,New York City,21.7,-0.3,36.1,13.7,757


## `4. Transform`

In [6]:
# present information about the data
df_info_summary = pd.DataFrame(columns=['data_attr', 'Data_Type', 'No_missing_values', 'Unique_Count'])
for column in df.columns:
    data_type = df[column].dtype
    missing_values = df[column].isna().sum()
    unique_count = df[column].nunique()
    df_info_summary = pd.concat([df_info_summary, pd.DataFrame({'data_attr': [column], 
                        'Data_Type': [data_type], 'No_missing_values': [missing_values],
                        'Unique_Count': [unique_count]})], ignore_index=True)
df_info_summary

,data_attr,Data_Type,No_missing_values,Unique_Count
0,Company_Name,object,0,81
1,Sector,object,0,81
2,Headquarters,object,0,57
3,Sales_in[$B],object,0,79
4,Profit_in[$B],object,0,58
5,Asset_in[$B],object,0,81
6,Market_Value_in[$B],float64,0,79
7,Overall_Rank,int64,0,80


In [7]:
#We have clean data and not much to do 
#create a new metrics -profit margin-ration of profit to sales and  to the df.
df['Sales_in[$B]'] = df['Sales_in[$B]'].astype(float)
df['Profit_in[$B]'] = df['Profit_in[$B]'].astype(float)
df['Profit_Margin_ratio[%]'] = (df['Profit_in[$B]']/df['Sales_in[$B]'] *100).round(2)
df.head()

,Company_Name,Sector,Headquarters,Sales_in[$B],Profit_in[$B],Asset_in[$B],Market_Value_in[$B],Overall_Rank,Profit_Margin_ratio[%]
0,WPP,Advertising,London,18.7,1.8,42.4,30.1,301,9.63
1,Boeing,Aerospace,Chicago,96.1,5.2,94.4,85.3,76,5.41
2,United Parcel Service,Air Courier,Sandy Springs,58.1,4.8,38.3,93.3,146,8.26
3,Delta Air Lines,Airline,Atlanta,40.5,4.7,53.3,34.4,170,11.60
4,Alcoa,Aluminum,New York City,21.7,-0.3,36.1,13.7,757,-1.38


In [8]:
# Save the dataframe as CSV
df.to_csv(tranformed_data_path, index=False) #'./Top_Companies_by_Sector_data.csv'

## `5. Load to the Database and SQL Querying as necessary`

In [9]:
df = pd.read_csv("Top_Companies_by_Sector_data.csv") #read transformed csv file to df
conn = sqlite3.connect('Global_Companies.db') #create connectionn to the new DB and it will be created locally

#create a table and connect to the database
table_name ='Top_Companies_by_Sector'
df.to_sql(table_name, conn, if_exists = 'replace', index =False) #if exists, fail or replace also exist
print("The table is ready to use")

The table is ready to use


In [10]:
#Run SQL queries using read_sql function in pandas
# View the first five rows of the table
query_statement1 = f"SELECT * FROM {table_name} LIMIT(5)"
query_output = pd.read_sql(query_statement1, conn)
query_output

,Company_Name,Sector,Headquarters,Sales_in[$B],Profit_in[$B],Asset_in[$B],Market_Value_in[$B],Overall_Rank,Profit_Margin_ratio[%]
0,WPP,Advertising,London,18.7,1.8,42.4,30.1,301,9.63
1,Boeing,Aerospace,Chicago,96.1,5.2,94.4,85.3,76,5.41
2,United Parcel Service,Air Courier,Sandy Springs,58.1,4.8,38.3,93.3,146,8.26
3,Delta Air Lines,Airline,Atlanta,40.5,4.7,53.3,34.4,170,11.60
4,Alcoa,Aluminum,New York City,21.7,-0.3,36.1,13.7,757,-1.38


In [11]:
# View Company name and its Headquarter for the five rows
query_statement2 = f"SELECT Company_Name, Headquarters From {table_name} LIMIT(5)"
query_output2 = pd.read_sql(query_statement2, conn)
query_output2

,Company_Name,Headquarters
0,WPP,London
1,Boeing,Chicago
2,United Parcel Service,Sandy Springs
3,Delta Air Lines,Atlanta
4,Alcoa,New York City


In [32]:
# View Company name and its Headquarter for the five rows
query_statement3 = f"SELECT Company_Name, Headquarters\
                    From {table_name} \
                    WHERE 'Profit_Margin_ratio[%]'>15"
query_output3 = pd.read_sql(query_statement3, conn)
query_output3

,Company_Name,Headquarters
0,WPP,London
1,Boeing,Chicago
2,United Parcel Service,Sandy Springs
3,Delta Air Lines,Atlanta
4,Alcoa,New York City
...,...,...
76,AT&T,Dallas
77,Aareal Bank,Wiesbaden
78,British American Tobacco,London
79,Mitsubishi Corp,Tokyo


In [12]:
# !pip install ipython-sql -install the library if it hasn't been install already

In [13]:
%load_ext sql
%sql sqlite:///Global_Companies.db
cur = conn.cursor() #if you want use to execute sql statement

In [14]:
%%sql 
SELECT * FROM Top_Companies_by_Sector LIMIT(5);


 * sqlite:///Global_Companies.db
Done.


Company_Name,Sector,Headquarters,Sales_in[$B],Profit_in[$B],Asset_in[$B],Market_Value_in[$B],Overall_Rank,Profit_Margin_ratio[%]
WPP,Advertising,London,18.7,1.8,42.4,30.1,301,9.63
Boeing,Aerospace,Chicago,96.1,5.2,94.4,85.3,76,5.41
United Parcel Service,Air Courier,Sandy Springs,58.1,4.8,38.3,93.3,146,8.26
Delta Air Lines,Airline,Atlanta,40.5,4.7,53.3,34.4,170,11.6
Alcoa,Aluminum,New York City,21.7,-0.3,36.1,13.7,757,-1.38


In [15]:
# confirm the number of rows in the table with df.shape 
%sql  SELECT COUNT(*) FROM Top_Companies_by_Sector;

 * sqlite:///Global_Companies.db
Done.


COUNT(*)
81


In [ ]:
conn.close()